In [1]:
import cv2
import torch
import os
from PIL import Image
import numpy as np

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'custom', '../models/yoloFire.pt')  # force_reload=True to update

# Other Parameters
pd = 0
text = "emergency"
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.8
font_thickness = 1
text_color = (255, 255, 255)  # White color
pixel_length_meters = 0.05

Using cache found in C:\Users\vikas/.cache\torch\hub\ultralytics_yolov5_master
C:\Users\vikas\Desktop\ForestFireTracking\envfire\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
YOLOv5  2024-5-25 Python-3.10.4 torch-2.2.1+cpu CPU

Fusing layers... 
custom_YOLOv5s summary: 232 layers, 7246518 parameters, 0 gradients
Adding AutoShape... 


In [3]:
def yolo(im, size=640):
    g = (size / max(im.size))  
    im = im.resize((int(x * g) for x in im.size), Image.LANCZOS)
    results = model(im)
    
#     results.render()  

    result_image = Image.fromarray(results.ims[0])
    result_frame = np.array(result_image)

    return results, result_frame

### 1. Area using pixel segmentation
- Output of this will be 3 frames

In [4]:
def preprocess1(results, result_frame):
    
#     black_frame = np.zeros_like(result_frame)
    black_frame = np.zeros((result_frame.shape[0] + 30, result_frame.shape[1], 3), dtype=np.uint8)
    for box in results.xyxy[0]:
        
        xmin, ymin, xmax, ymax, _, _ = box
        
        cv2.rectangle(result_frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (10, 255, 0), 2)
        cv2.imshow('Result Frame', result_frame)
        
        # Extract the region of interest (ROI) from the original frame
        roi = result_frame[int(ymin)-pd:int(ymax)+pd, int(xmin)-pd:int(xmax)+pd]
        
        # Convert the ROI to YCbCr color space
        roi_ycbcr = cv2.cvtColor(roi, cv2.COLOR_RGB2YCrCb)
        
        roi_highlighted = np.zeros_like(roi)
        for y in range(roi.shape[0]):
            for x in range(roi.shape[1]):
                Y, Cr, Cb = roi_ycbcr[y, x]
                if Y >= Cb and Cr >= Cb and Y <= 150:
#                     fire_pixels = fire_pixels + 1
                    
                    roi_highlighted[y, x] = np.array([0,143,255]) 
#                     roi_highlighted[y,x] = roi[y, x][::-1]

        black_frame[int(ymin)-pd:int(ymax)+pd, int(xmin)-pd:int(xmax)+pd] = roi_highlighted
        
        area = 0
        gray = cv2.cvtColor(black_frame, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)
        
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Iterate through each contour
        for contour in contours:
            # Calculate area of the contour
            pixel_area = cv2.contourArea(contour)  # Area in pixels

            area += pixel_area
            # Draw contour on the original image
            cv2.drawContours(black_frame, [contour], -1, (0, 255, 0), 1)
        
        pr = area*100/(black_frame.shape[0]*black_frame.shape[1])
        
        cv2.putText(black_frame, "Area: {}%".format(round(pr,2)), (20, 370), font, font_scale, text_color, font_thickness, lineType=cv2.LINE_AA)
    
    return black_frame

### 2. Area using Bounding boxes
- This will show 2 frames (Video & Bounding box)

In [7]:
def preprocess2(results, result_frame):
    total_area = 0
    black_frame = np.zeros((result_frame.shape[0] + 30, result_frame.shape[1], 3), dtype=np.uint8)
    for box in results.xyxy[0]:
        
        xmin, ymin, xmax, ymax, _, _ = box
        cv2.rectangle(black_frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (10, 255, 0), 1)
        
        w = xmax - xmin
        h = ymax - ymin
        
        total_area += w * h

        # Calculate the percentage of the frame that the average area occupies
        pr =  total_area* 100 / (black_frame.shape[0] * black_frame.shape[1])

        cv2.putText(black_frame, "Area: {}%".format(round(float(pr), 2)), (20, 370), font, font_scale, text_color, font_thickness, lineType=cv2.LINE_AA)

    return black_frame

In [12]:
areas = []
# start_frame = 6100  # Change this to the frame number you want to start from

cap = cv2.VideoCapture("C:/Users/vikas/Downloads/1-Zenmuse_X4S_1.mp4")
# cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

while True:
    # Read a frame from the video
    ret, frame = cap.read()
    
    # Break the loop if the video is over
    if not ret:
        print('Error in capturing frames')
        break
    
    # Convert the frame from OpenCV format (BGR) to PIL format (RGB)
    frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    # Function Calling
    results, result_frame = yolo(frame_pil)
    
#     black_frame = preprocess1(results, result_frame)
    
    black_frame = preprocess2(results, result_frame)
    
    cv2.imshow('YOLOv5 result', cv2.cvtColor(result_frame, cv2.COLOR_RGB2BGR))
    
    # Detected Area After Pre-processing
    cv2.imshow('Preprocess', black_frame)
    
    # Press q to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [12]:
cap.release()
cv2.destroyAllWindows()

### Farneback Method for Optical flow!

In [21]:
import cv2
import numpy as np

color = [255, 255, 255]
avgx = []
avgy = []

# Clockwise positive direction

# Open video file
cap = cv2.VideoCapture('../demoVideos/paperFire1.mp4')

# cap = cv2.VideoCapture('../DemoVideos/paperFire2.mp4')
# cap.set(cv2.CAP_PROP_POS_FRAMES, 1000)

# Create random colors for visualizing optical flow tracks
# color = np.random.randint(0,255,(100,3))

ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

hsv = np.zeros_like(frame1)
hsv[...,1] =  255  # Assigning 255 to Saturation Value in hsv Image

# Create a black frame for motion visualization
black_frame = np.zeros_like(frame1)

frame_count = 1

while True:
    ret, frame2 = cap.read()
    
#     frame_count += 1
    
    if ret and frame_count:
#         frame_count = 0
        new = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        # Calculate Optical Flow
        flow = cv2.calcOpticalFlowFarneback(prvs, new, None, 
                                            0.5, 3, 15, 3, 5, 1.2, 0)

        # Get average flow direction
        avgx.append(np.mean(flow[..., 0]))
        avgy.append(np.mean(flow[..., 1]))
        
        if len(avgx) > 30: 
            avg_fx = np.mean(avgx)
            avg_fy = np.mean(avgy)
            avg_direction_angle = np.arctan2(avg_fy, avg_fx)
            avg_direction_degrees = np.degrees(avg_direction_angle)
            
            # Set text on black_frame
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame2, f"Direction wrt X+: {avg_direction_degrees:.2f} degrees", (10, 30), font, 1, (0, 255, 0), 1, cv2.LINE_AA)
            
            avgx.pop()
            avgy.pop()

        # Overlay motion visualization onto the original frame
        for y in range(0, black_frame.shape[0], 10):
            for x in range(0, black_frame.shape[1], 10):
                fx, fy = flow[y, x]
                cv2.line(black_frame, (x, y), (int(x + fx), int(y + fy)), color, 1)
        
    
        color[0] -= 1
        color[1] -= 1

        # Display original video and motion visualization
        cv2.imshow('Original Video', frame2)
        cv2.imshow('Fire Spread Motion', black_frame)
    
        prvs = new
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break   

# Release video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


### Motion Analysis by subtracting frames!

In [11]:
import cv2
import numpy as np

# Load the video
video_path = '../demoVideos/forestFire1.mp4'
cap = cv2.VideoCapture(video_path)

# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Initialize the background subtractor
fgbg = cv2.createBackgroundSubtractorMOG2()

while True:
    # Read a frame from the video
    ret, frame = cap.read()

    # Check if the frame was read successfully
    if not ret:
        print("Error in capturing frames")
        break
    
    # Apply background subtraction to detect fire pixels
    fgmask = fgbg.apply(frame)
    
    cv2.imshow('Mask', fgmask)

    # Apply thresholding to segment fire regions
    _, thresh = cv2.threshold(fgmask, 150, 250, cv2.THRESH_BINARY)
    

    # Find contours of fire regions
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw bounding boxes around fire regions
    for contour in contours:
        area = cv2.contourArea(contour)
        
        if area > 500:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

    # Display the frame with fire detection
    cv2.imshow('Fire Detection', frame)

    # Check for user input to exit
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


### Direction using Lucas Kanade > You can skip this :)

In [12]:
import cv2
import numpy as np

cap = cv2.VideoCapture('../DemoVideos/forestFire1.mp4')
# cap.set(cv2.CAP_PROP_POS_FRAMES, 1000)

feature_params = dict(maxCorners=2500, qualityLevel=0.5, minDistance=5, blockSize=2)

lk_params = dict(winSize=(40,40), maxLevel=5, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 20, 0.03))

color = np.random.randint(0, 255, (100, 3))

ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

mask = np.zeros_like(old_frame)

frame_count = 0

while True:
    ret, frame = cap.read()
 
    if not ret:
        break
    
    frame_count += 1
    
    if frame_count % 50 == 0:  
        frame_count = 0
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate Optical Flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, gray, p0, None, **lk_params)

        # Select good points
        good_new = p1[st==1]
        good_old = p0[st==1]

        # Draw the optical flow tracks
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
#             frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)

        # Combine frame with optical flow tracks
#         img = cv2.add(frame, mask)
        
        # Display the result
        cv2.imshow('Optical Flow', mask)
        
        # Update old frame and points for next iteration
        old_gray = gray.copy()
        p0 = good_new.reshape(-1, 1, 2)
        
    cv2.imshow('Frame', frame)
    
    # Check for user input to exit
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [160]:
cap.release()
cv2.destroyAllWindows()

In [27]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('../models/yoloNano.pt', verbose=False)

# Define the video path and starting frame
video_path = "../demoVideos/1-Zenmuse_X4S_Clip2.mp4"
start_frame = 500

# Open the video file and set the starting frame
cap = cv2.VideoCapture(video_path)
cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()
    cnt += 1
    

    if success:
        frame = cv2.resize(frame, (960, 540))
        
        cnt = 0
        # Run YOLOv8 inference on the frame
        results = model(frame)

        # Extract the detection results
        detections = results[0].obb.xywhr.cpu().numpy()  # Bounding box coordinates
#         scores = results[0].obb.conf.cpu().numpy()  # Confidence scores
#         classes = results[0].obb.cls.cpu().numpy()  # Class indices

        # Draw custom bounding boxes on the frame
        for i in range(len(detections)):
            x, y, w, h, r = detections[i]
#             score = scores[i]
#             class_idx = int(classes[i])

            # Convert the center (x, y), width, height, and rotation angle into a box format
            rect = ((x, y), (w, h), np.degrees(r))
            box = cv2.boxPoints(rect)
            box = np.intp(box)

            # Draw the bounding box
#             color = colors[class_idx % len(colors)]
#             label = f"{labels[class_idx % len(labels)]}: {score:.2f}"
            cv2.polylines(frame, [box], True, (0,0,255), 2)
#             cv2.putText(frame, label, (int(box[0][0]), int(box[0][1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Display the annotated frame
        cv2.imshow("YOLOv8 Inference", frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()



0: 384x640 298.8ms
Speed: 31.1ms preprocess, 298.8ms inference, 16.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 74.3ms
Speed: 2.8ms preprocess, 74.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 69.3ms
Speed: 10.8ms preprocess, 69.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 90.0ms
Speed: 0.0ms preprocess, 90.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 80.5ms
Speed: 0.0ms preprocess, 80.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 76.3ms
Speed: 0.0ms preprocess, 76.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 94.1ms
Speed: 0.0ms preprocess, 94.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 70.9ms
Speed: 2.0ms preprocess, 70.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 73.5ms
Speed: 2.0ms preprocess, 73.5ms inference, 2.0ms


0: 384x640 78.0ms
Speed: 0.0ms preprocess, 78.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 75.8ms
Speed: 0.0ms preprocess, 75.8ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 69.2ms
Speed: 2.0ms preprocess, 69.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 81.2ms
Speed: 0.0ms preprocess, 81.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 64.2ms
Speed: 2.1ms preprocess, 64.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 76.1ms
Speed: 2.0ms preprocess, 76.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 62.1ms
Speed: 2.1ms preprocess, 62.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 78.8ms
Speed: 0.0ms preprocess, 78.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 61.0ms
Speed: 4.5ms preprocess, 61.0ms inference, 0.0ms post


0: 384x640 76.9ms
Speed: 0.0ms preprocess, 76.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 86.2ms
Speed: 0.0ms preprocess, 86.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 78.1ms
Speed: 0.0ms preprocess, 78.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 71.7ms
Speed: 0.0ms preprocess, 71.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 80.9ms
Speed: 0.0ms preprocess, 80.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 76.9ms
Speed: 0.0ms preprocess, 76.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 82.6ms
Speed: 0.0ms preprocess, 82.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 66.1ms
Speed: 10.5ms preprocess, 66.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 81.2ms
Speed: 0.0ms preprocess, 81.2ms inference, 0.0ms pos


0: 384x640 90.7ms
Speed: 3.8ms preprocess, 90.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 83.3ms
Speed: 7.8ms preprocess, 83.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 96.0ms
Speed: 0.0ms preprocess, 96.0ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 105.1ms
Speed: 1.4ms preprocess, 105.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 76.0ms
Speed: 0.0ms preprocess, 76.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 250.4ms
Speed: 1.3ms preprocess, 250.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 104.8ms
Speed: 5.3ms preprocess, 104.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 116.8ms
Speed: 2.2ms preprocess, 116.8ms inference, 4.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 93.6ms
Speed: 4.5ms preprocess, 93.6ms inference, 0.


0: 384x640 125.6ms
Speed: 0.0ms preprocess, 125.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 109.6ms
Speed: 0.0ms preprocess, 109.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 148.4ms
Speed: 0.0ms preprocess, 148.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 160.9ms
Speed: 3.3ms preprocess, 160.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 96.0ms
Speed: 10.5ms preprocess, 96.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 93.1ms
Speed: 0.0ms preprocess, 93.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 90.1ms
Speed: 0.0ms preprocess, 90.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 99.7ms
Speed: 2.0ms preprocess, 99.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 89.9ms
Speed: 0.0ms preprocess, 89.9ms inference, 0

...


0: 384x640 97.9ms
Speed: 0.0ms preprocess, 97.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 105.4ms
Speed: 0.0ms preprocess, 105.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 89.6ms
Speed: 0.0ms preprocess, 89.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 115.8ms
Speed: 8.5ms preprocess, 115.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 107.8ms
Speed: 0.0ms preprocess, 107.8ms inference, 3.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 76.9ms
Speed: 2.7ms preprocess, 76.9ms inference, 15.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 110.1ms
Speed: 2.3ms preprocess, 110.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 92.2ms
Speed: 0.0ms preprocess, 92.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 84.9ms
Speed: 0.0ms preprocess, 84.9ms inference, 0


0: 384x640 128.1ms
Speed: 0.0ms preprocess, 128.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 109.2ms
Speed: 0.0ms preprocess, 109.2ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 111.0ms
Speed: 0.0ms preprocess, 111.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 84.6ms
Speed: 0.0ms preprocess, 84.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 110.3ms
Speed: 0.0ms preprocess, 110.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 115.4ms
Speed: 0.0ms preprocess, 115.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 116.9ms
Speed: 0.0ms preprocess, 116.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 97.0ms
Speed: 0.0ms preprocess, 97.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 110.7ms
Speed: 0.0ms preprocess, 110.7ms inferen


0: 384x640 (no detections), 88.3ms
Speed: 0.0ms preprocess, 88.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 128.7ms
Speed: 0.0ms preprocess, 128.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 100.0ms
Speed: 0.0ms preprocess, 100.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 94.2ms
Speed: 0.0ms preprocess, 94.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 94.4ms
Speed: 0.0ms preprocess, 94.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 103.0ms
Speed: 0.0ms preprocess, 103.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 110.3ms
Speed: 15.6ms preprocess, 110.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 93.0ms
Speed: 16.2ms preprocess


0: 384x640 (no detections), 128.3ms
Speed: 0.0ms preprocess, 128.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 97.3ms
Speed: 5.9ms preprocess, 97.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 126.2ms
Speed: 0.0ms preprocess, 126.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 99.9ms
Speed: 0.0ms preprocess, 99.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 110.6ms
Speed: 12.5ms preprocess, 110.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 78.7ms
Speed: 0.0ms preprocess, 78.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 86.2ms
Speed: 0.0ms preprocess, 86.2ms inference, 12.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 94.4ms
Speed: 14.4ms preprocess, 94.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 141.3ms
Speed: 0.


0: 384x640 122.4ms
Speed: 4.1ms preprocess, 122.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 80.5ms
Speed: 12.9ms preprocess, 80.5ms inference, 16.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 75.2ms
Speed: 0.0ms preprocess, 75.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 94.2ms
Speed: 0.0ms preprocess, 94.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 94.0ms
Speed: 0.0ms preprocess, 94.0ms inference, 6.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 80.7ms
Speed: 0.0ms preprocess, 80.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 69.4ms
Speed: 10.3ms preprocess, 69.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 64.4ms
Speed: 7.5ms preprocess, 64.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 82.4ms
Speed: 11.0ms preprocess, 82.4ms inference, 0.0m

In [14]:
# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()